In [1]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS

from langchain_text_splitters import (
    TokenTextSplitter, 
    RecursiveCharacterTextSplitter
)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_chroma import Chroma
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.chains import (
    RetrievalQAWithSourcesChain, 
    RetrievalQA
)
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers.merger_retriever import MergerRetriever

from dotenv import load_dotenv, find_dotenv
import os
from langchain.chains import RetrievalQA
import langchain
langchain.debug = False


load_dotenv(
    find_dotenv()
)

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model_name = [
    "llama3-70b-8192", 
    "gemma2-9b-it", 
    "qwen-qwq-32b"
]

llm_model = ChatGroq(
    model=model_name[1], 
    temperature=0.4, 
    max_tokens=512, 
    model_kwargs={
        "top_p" : 0.9,
    },
)


def printDocuments(docs) :
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {index + 1}:- \n\n" + d.page_content for index, d in enumerate(docs)
            ]
        )
    )

In [3]:
def DataAugmentationWithDualRetriever(
    data_path, embedding_provider, chunk_size, chunk_overlap, 
    faiss_retriever_search_type, faiss_retriever_k_documents, 
    chroma_retriever_search_type, chroma_retriever_k_documents, **kwargs) :
    if kwargs["execute_function"] :
        print("\n[INFO] ----> Running the Data Augmentation Pipeline for RAG, please wait....\n")
    
        documents = PyMuPDFLoader(
            data_path,
        ).load()
        print("\n[INFO] ----> Total Pages in the original document  are :- {}".format(len(documents)))

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

        document_chunks = splitter.split_documents(
            documents
        )

        print("[INFO] ----> Total Document Chunks Created are :- {}\n".format(len(document_chunks)))

        if embedding_provider == "Google" :
            print("[INFO] ----> Using the Google-AI Embedding model...")
            embeddings = GoogleGenerativeAIEmbeddings(
                model="models/embedding-001"
            )
            print("[INFO] ----> Google-AI Embedding model loaded.\n")

        elif embedding_provider == "OpenAI" :
            print("[INFO] ----> Using the Open-AI Embedding model....")
            embeddings = OpenAIEmbeddings()
            print("[INFO] ----> OpenAI Embedding model loaded.\n")

        print("[INFO] ----> Creating the FAISS Vectorstore, please wait.....")
        faissRetriever = FAISS.from_documents(
            document_chunks, embeddings).as_retriever(
                search_type=faiss_retriever_search_type, 
                search_kwargs={
                    "k" : faiss_retriever_k_documents
                }
            )
        print("[INFO] ----> FAISS Retriever Created successfully.....\n")

        print("[INFO] ----> Creating the Chroma Vectorstore, please wait.....")
        chromaRetriever = Chroma.from_documents(
            document_chunks, 
            embeddings,
        ).as_retriever(
            search_type=chroma_retriever_search_type, 
            search_kwargs={
                "k" : chroma_retriever_k_documents, 
            }
        )
        print("[INFO] ----> Chroma Retriever Created successfully.....\n")
        print("[INFO] ----> RAG Data Augmentation completed......")

        return faissRetriever, chromaRetriever, embeddings
    else :
        print("[INFO] ----> Change the execute_function argument to True and then run...")
        return None, None


def EnsembleContextualCompressionRetriever(**kwargs) :
    if kwargs["execute_pipeline"] :
        redundant_filter = EmbeddingsRedundantFilter(
            embeddings=kwargs['embeddings'],
        )
        reordering = LongContextReorder()

        relevant_filter_faiss = EmbeddingsFilter(
            embeddings=kwargs['embeddings'], 
            similarity_threshold=kwargs['faiss_embeddingfilter_threshold'],
        )
        relevant_filter_chroma = EmbeddingsFilter(
            embeddings=kwargs['embeddings'], 
            similarity_threshold=kwargs['chroma_embeddingfilter_threshold'],
        )

        document_pipeline_compressor_faiss = DocumentCompressorPipeline(
            transformers=[
                redundant_filter, 
                relevant_filter_faiss, 
                reordering,
            ]
        )
        document_pipeline_compressor_chroma = DocumentCompressorPipeline(
            transformers=[
                redundant_filter, 
                relevant_filter_chroma,
                reordering,
            ]
        )

        # First Compression Retriever with FAISS
        compression_retriever_with_faiss = ContextualCompressionRetriever(
            base_compressor=document_pipeline_compressor_faiss, 
            base_retriever=kwargs['faissRetriever'],

        )
        # Second Compression Retriever with Chroma
        compression_retriever_with_chroma = ContextualCompressionRetriever(
            base_compressor=document_pipeline_compressor_chroma, 
            base_retriever=kwargs['chromaRetriever'],
        )

        lotrCompression = MergerRetriever(
            retrievers=[
                compression_retriever_with_faiss, 
                compression_retriever_with_chroma
            ]
        )
        return lotrCompression
    else :
        print("[INFO] ----> Change the execute_function argument to True and then run...")
        return None


def generateAnswerFunction(
    question, llm_model, retriever_function, 
    verbose=-1, **kwargs) :
    if kwargs["parse_function"] :
        rag_pipeline_type = kwargs["ragPipelineConfig"]
        if rag_pipeline_type == "RetrievalQAWithSourcesChain" :
            print("[INFO] ----> Running the RetrievalQAWithSourcesChain Pipeline.....\n")
            rag_pipeline = RetrievalQAWithSourcesChain.from_chain_type(
                llm = llm_model,
                chain_type = kwargs["chain_type"], 
                retriever = retriever_function,
                return_source_documents=True
            )
            print("[INFO] ----> Generating the answer for:- \nQuestion:- {}".format(question))
            generated_answer = rag_pipeline.invoke(question)
            result = generated_answer["answer"]
            print("[INFO] ----> Result Answer generated.....\n")
            if verbose > -1 :
                print("Result :- {}{}".format("\n", result))
            return result

        elif rag_pipeline_type == "RetrievalQAChain" :
            print("[INFO] ----> Running the RetrievalQAChain Pipeline.....\n")
            rag_pipeline = RetrievalQA.from_chain_type(
                llm = llm_model,
                chain_type = kwargs["chain_type"], 
                retriever = retriever_function,
                return_source_documents=True
            )
            print("[INFO] ----> Generating the answer for:- \nQuestion:- {}".format(question))
            generated_answer = rag_pipeline.invoke(question)
            result = generated_answer["result"]
            print("[INFO] ----> Result Answer generated.....\n")
            if verbose > -1 :
                print("Result :- {}{}".format("\n", result))
            return result



dataPath = "Data\\ReAct.pdf"
embedding = "Google"

faissRetriever, chromaRetriever, embeddings = DataAugmentationWithDualRetriever(
    data_path = dataPath,
    embedding_provider = embedding,
    chunk_size = 512,
    chunk_overlap = 128,
    faiss_retriever_search_type = "similarity",
    faiss_retriever_k_documents = 3,
    chroma_retriever_search_type = "similarity",
    chroma_retriever_k_documents = 4,
    execute_function = True
)

retriever = EnsembleContextualCompressionRetriever(
    faiss_embeddingfilter_threshold = 0.75,
    chroma_embeddingfilter_threshold = 0.6,
    embeddings=embeddings, 
    execute_pipeline=True,
    faissRetriever=faissRetriever, 
    chromaRetriever=chromaRetriever,
)



[INFO] ----> Running the Data Augmentation Pipeline for RAG, please wait....


[INFO] ----> Total Pages in the original document  are :- 33
[INFO] ----> Total Document Chunks Created are :- 293

[INFO] ----> Using the Google-AI Embedding model...
[INFO] ----> Google-AI Embedding model loaded.

[INFO] ----> Creating the FAISS Vectorstore, please wait.....
[INFO] ----> FAISS Retriever Created successfully.....

[INFO] ----> Creating the Chroma Vectorstore, please wait.....
[INFO] ----> Chroma Retriever Created successfully.....

[INFO] ----> RAG Data Augmentation completed......


In [4]:
query = "Explain in details about the Chain of thought prompting as mentioned in ReAct Paper."

answer = generateAnswerFunction(
    question=query,
    llm_model=llm_model, 
    retriever_function=retriever,
    parse_function=True, 
    chain_type="stuff",

    # Choices are "RetrievalQAWithSourcesChain" or "RetrievalQAChain"
    ragPipelineConfig="RetrievalQAWithSourcesChain"
)

print("\n")
print("===" * 100)
print("Final Answer :- {}{}".format("\n", answer))


[INFO] ----> Running the RetrievalQAWithSourcesChain Pipeline.....

[INFO] ----> Generating the answer for:- 
Question:- Explain in details about the Chain of thought prompting as mentioned in ReAct Paper.
[INFO] ----> Result Answer generated.....



Final Answer :- 
Chain-of-thought prompting, as mentioned in the ReAct paper, involves providing the model with a sequence of thoughts leading to a solution. 

The paper describes it as a "static black box" because while the model uses chain-of-thought reasoning, the process is not transparent. We don't know exactly how the model arrives at each step in the chain.

The ReAct paper proposes a new prompting method called "ReAct" that builds upon chain-of-thought prompting by incorporating both reasoning (thoughts) and actions taken by the model. This allows for a more dynamic and interpretable understanding of the model's decision-making process.


